In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt, floor, log10, log


def round_sig(x: float, sig: int) -> float:
    """Rounds x to sig significant digits"""
    return x if x == 0 else round(x, sig - int(floor(log10(abs(x)))) - 1)


plt.rcParams["figure.figsize"] = (20, 28)
# plt.style.use('dark_background')


### Ładowanie danych

In [ ]:
df = pd.read_csv("data.csv")
df.head()

U_heating = df["U"][:11]  # in Volts
U_cooling = df["U"][11:]

R_1_heating = df["R1"][:11]  # in Kilo Ohm
R_1_cooling = df["R1"][11:]

R_2_heating = df["R2"][:11]  # in Kilo Ohm
R_2_cooling = df["R2"][11:]

# 0.01V - 1 celcius
T_heating = [round_sig(x / 0.01, 3) for x in U_heating]
T_cooling = [round_sig(x / 0.01, 3) for x in U_cooling]

U_heating_error = [round_sig((0.005 * x + 1.0 * 0.001) / sqrt(3), 2) for x in U_heating]
U_cooling_error = [round_sig((0.005 * x + 1.0 * 0.001) / sqrt(3), 2) for x in U_cooling]

R_1_heating_error = [
    round_sig((0.015 * x + 3.0 * 0.01) / sqrt(3), 2) for x in R_1_heating
]
R_2_heating_error = [
    round_sig((0.008 * x + 1.0 * 0.01) / sqrt(3), 2) for x in R_2_heating
]

R_1_cooling_error = [
    round_sig((0.015 * x + 3.0 * 0.01) / sqrt(3), 2) for x in R_1_cooling
]
R_2_cooling_error = [
    round_sig((0.008 * x + 1.0 * 0.01) / sqrt(3), 2) for x in R_2_cooling
]

T_heating_error = [round_sig(x / 0.01, 2) for x in U_heating_error]
T_cooling_error = [round_sig(x / 0.01, 2) for x in U_cooling_error]


### Wykres $R_1(T)$ i $R_2(T)$ dla ogrzewania

In [ ]:
plt.errorbar(
    T_heating,
    R_1_heating,
    xerr=T_heating_error,
    yerr=R_1_heating_error,
    fmt="o",
    label="xd",
    linewidth=1,
    markersize=3,
    color="red",
)

plt.errorbar(
    T_heating,
    R_2_heating,
    xerr=T_heating_error,
    yerr=R_2_heating_error,
    fmt="o",
    label="xd",
    linewidth=1,
    markersize=3,
    color="blue",
)

plt.title(
    r"Wykres $R(T)$ dla ogrzewania termistorów",
    fontsize=30,
)
plt.tick_params(axis="both", which="major", labelsize=20)
plt.ylabel(r"$R$, $10^3 \cdot \Omega$", fontsize=30)
plt.xlabel(r"$T$, $^{\circ}C$", fontsize=30)

plt.savefig("./img/wykres1.jpg")


### Wykres $R_1(T)$ i $R_2(T)$ dla ochładzania

In [ ]:
plt.errorbar(
    T_cooling,
    R_1_cooling,
    xerr=T_cooling_error,
    yerr=R_1_cooling_error,
    fmt="o",
    label="xd",
    linewidth=1,
    markersize=3,
    color="red",
)

plt.errorbar(
    T_cooling,
    R_2_cooling,
    xerr=T_cooling_error,
    yerr=R_2_cooling_error,
    fmt="o",
    label="xd",
    linewidth=1,
    markersize=3,
    color="blue",
)

plt.title(
    r"Wykres $R(T)$ dla ochładzania termistorów",
    fontsize=30,
)
plt.tick_params(axis="both", which="major", labelsize=20)
plt.ylabel(r"$R$, $10^3 \cdot \Omega$", fontsize=30)
plt.xlabel(r"$T$, $^{\circ}C$", fontsize=30)

plt.savefig("./img/wykres2.jpg")


### Wykres $f(\frac{1}{T}) = \ln(R)$ dla ogrzewania

In [ ]:
T_inverse_heating = np.array([round_sig(1 / (x + 273.15), 3) for x in T_heating])
R_1_ln_heating = [round_sig(log(x), 3) for x in R_1_heating]
R_2_ln_heating = [round_sig(log(x), 3) for x in R_2_heating]

# niepewnosc w celcjuszach to to samo co w kelwinach

T_inverse_heating_error = [
    round_sig(u_T / (t + 273.15) ** 2, 2) for u_T, t in zip(T_heating_error, T_heating)
]

R_1_ln_heating_error = [
    round_sig(u_R / R, 2) for u_R, R in zip(R_1_heating_error, R_1_heating)
]

R_2_ln_heating_error = [
    round_sig(u_R / R, 2) for u_R, R in zip(R_2_heating_error, R_2_heating)
]

plt.errorbar(
    T_inverse_heating,
    R_1_ln_heating,
    xerr=T_inverse_heating_error,
    yerr=R_1_ln_heating_error,
    fmt="o",
    label="xd",
    linewidth=1,
    markersize=3,
    color="red",
)

plt.errorbar(
    T_inverse_heating,
    R_2_ln_heating,
    xerr=T_inverse_heating_error,
    yerr=0,
    fmt="o",
    label="xd",
    linewidth=1,
    markersize=3,
    color="blue",
)

# regresja liniowa dla R_1
a_1_heating, b_1_heating = np.polyfit(T_inverse_heating, R_1_ln_heating, 1)
a_1_heating = round_sig(a_1_heating, 3)
b_1_heating = round_sig(b_1_heating, 3)
plt.plot(
    T_inverse_heating, a_1_heating * T_inverse_heating + b_1_heating, color="orange"
)

# regresja liniowa dla R_2
a_2_heating, b_2_heating = np.polyfit(T_inverse_heating, R_2_ln_heating, 1)
a_2_heating = round_sig(a_2_heating, 3)
b_2_heating = round_sig(b_2_heating, 3)
plt.plot(
    T_inverse_heating,
    a_2_heating * T_inverse_heating + b_2_heating,
    color="midnightblue",
)


def get_a_error(X, Y, a: float, b: float) -> float:
    """Zwraca niepewność współczynnika kierukowego a"""

    n = len(X)
    S_x = sum(X)
    S_xx = sum([x * x for x in X])
    S_ee = sum([(y - a * x - b) ** 2 for x, y in zip(X, Y)])
    return sqrt(n / (n - 2) * S_ee / (n * S_xx - S_x * S_x))


def get_b_error(X, Y, a: float, b: float) -> float:
    """Zwraca niepewność wyrazu wolnego b"""

    n = len(X)
    S_x = sum(X)
    S_xx = sum([x * x for x in X])
    S_ee = sum([(y - a * x - b) ** 2 for x, y in zip(X, Y)])
    return sqrt(1 / (n - 2) * (S_xx * S_ee) / (n * S_xx - S_x * S_x))


a_1_heating_error = round_sig(
    get_a_error(T_inverse_heating, R_1_ln_heating, a_1_heating, b_1_heating), 2
)
b_1_heating_error = round_sig(
    get_b_error(T_inverse_heating, R_1_ln_heating, a_1_heating, b_1_heating), 2
)

a_2_heating_error = round_sig(
    get_a_error(T_inverse_heating, R_2_ln_heating, a_2_heating, b_2_heating), 2
)
b_2_heating_error = round_sig(
    get_b_error(T_inverse_heating, R_2_ln_heating, a_2_heating, b_2_heating), 2
)

print(f"a1={a_1_heating}, u(a1)={a_1_heating_error}")
print(f"b1={b_1_heating}, u(b1)={b_1_heating_error}")
print(f"a2={a_2_heating}, u(a2)={a_2_heating_error}")
print(f"b2={b_2_heating}, u(b2)={b_2_heating_error}")

plt.title(
    r"Wykres $f(\frac{1}{T}) = \ln(R)$ dla ogrzewania termistorów",
    fontsize=30,
)
plt.tick_params(axis="both", which="major", labelsize=20)
plt.ylabel(r"$\ln(R)$, 1", fontsize=30)
plt.xlabel(r"$\frac{1}{T}$, $\frac{1}{K}$", fontsize=30)

plt.savefig("./img/wykres3.jpg")


### Wykres $f(\frac{1}{T}) = \ln(R)$ dla ochładzania

In [ ]:
T_inverse_cooling = np.array([round_sig(1 / (x + 273.15), 3) for x in T_cooling])
R_1_ln_cooling = [
    round_sig(log(x), 3) for x in R_1_cooling
]  # dobra kurwa to bylo problemem a nie dane xd
R_2_ln_cooling = [round_sig(log(x), 3) for x in R_2_cooling]

T_inverse_cooling_error = [
    round_sig(u_T / (t + 273.15) ** 2, 2) for u_T, t in zip(T_cooling_error, T_cooling)
]

R_1_ln_cooling_error = [
    round_sig(u_R / R, 2) for u_R, R in zip(R_1_cooling_error, R_1_cooling)
]

R_2_ln_cooling_error = [
    round_sig(u_R / R, 2) for u_R, R in zip(R_2_cooling_error, R_2_cooling)
]

plt.errorbar(
    T_inverse_cooling,
    R_1_ln_cooling,
    xerr=T_inverse_cooling_error,
    yerr=R_1_ln_cooling_error,
    fmt="o",
    label="xd",
    linewidth=1,
    markersize=3,
    color="red",
)

plt.errorbar(
    T_inverse_cooling,
    R_2_ln_cooling,
    xerr=T_inverse_cooling_error,
    yerr=R_2_ln_cooling_error,
    fmt="o",
    label="xd",
    linewidth=1,
    markersize=3,
    color="blue",
)

# regresja liniowa dla R_1
a_1_cooling, b_1_cooling = np.polyfit(T_inverse_cooling, R_1_ln_cooling, 1)
a_1_cooling = round_sig(a_1_cooling, 3)
b_1_cooling = round_sig(b_1_cooling, 3)
plt.plot(
    T_inverse_cooling, a_1_cooling * T_inverse_cooling + b_1_cooling, color="orange"
)

# regresja liniowa dla R_2
a_2_cooling, b_2_cooling = np.polyfit(T_inverse_cooling, R_2_ln_cooling, 1)
a_2_cooling = round_sig(a_2_cooling, 3)
b_2_cooling = round_sig(b_2_cooling, 3)
plt.plot(
    T_inverse_cooling,
    a_2_cooling * T_inverse_cooling + b_2_cooling,
    color="midnightblue",
)

a_1_cooling_error = round_sig(
    get_a_error(T_inverse_cooling, R_1_ln_cooling, a_1_cooling, b_1_cooling), 2
)
b_1_cooling_error = round_sig(
    get_b_error(T_inverse_cooling, R_1_ln_cooling, a_1_cooling, b_1_cooling), 2
)

a_2_cooling_error = round_sig(
    get_a_error(T_inverse_cooling, R_2_ln_cooling, a_2_cooling, b_2_cooling), 2
)
b_2_cooling_error = round_sig(
    get_b_error(T_inverse_cooling, R_2_ln_cooling, a_2_cooling, b_2_cooling), 2
)

print(f"a1={a_1_cooling}, u(a1)={a_1_cooling_error}")
print(f"b1={b_1_cooling}, u(b1)={b_1_cooling_error}")
print(f"a2={a_2_cooling}, u(a2)={a_2_cooling_error}")
print(f"b2={b_2_cooling}, u(b2)={b_2_cooling_error}")

plt.title(
    r"Wykres $f(\frac{1}{T}) = \ln(R)$ dla ochładzania termistorów",
    fontsize=30,
)
plt.tick_params(axis="both", which="major", labelsize=20)
plt.ylabel(r"$\ln(R)$, 1", fontsize=30)
plt.xlabel(r"$\frac{1}{T}$, $\frac{1}{K}$", fontsize=30)

plt.savefig("./img/wykres4.jpg")


### Wyznacznie szerokości przerw energetycznych $\Delta E$ dla obu termistorów 

In [ ]:
# ln(R) = 1/T * a + b
# a = dE/2k, b = ln(R0)
# dE = a * 2 * k

# a = ln(R) * K
joule_to_electronvolt = 6.242 * pow(10, 18)
boltzmann_constant = 1.381 * pow(10, -23)  # Joule / Kelvin

# Ocieplanie
E_1_heating = round_sig(a_1_heating * 2 * boltzmann_constant * joule_to_electronvolt, 3)
E_2_heating = round_sig(a_2_heating * 2 * boltzmann_constant * joule_to_electronvolt, 3)

print("Ocieplanie")
print(E_1_heating, E_2_heating)

# Ochładzanie
E_1_cooling = round_sig(a_1_cooling * 2 * boltzmann_constant * joule_to_electronvolt, 3)
E_2_cooling = round_sig(a_2_cooling * 2 * boltzmann_constant * joule_to_electronvolt, 3)

print("Ochladzanie")
print(E_1_cooling, E_2_cooling)


### Niepewność $u(\Delta E)$

In [ ]:
# dE = 2 * a * k
# u(dE) = 2*k*u(a)

# Ocieplanie
E_1_heating_error = round_sig(
    2 * boltzmann_constant * a_1_heating_error * joule_to_electronvolt, 2
)
E_2_heating_error = round_sig(
    2 * boltzmann_constant * a_2_heating_error * joule_to_electronvolt, 2
)

print("Ocieplanie")
print(E_1_heating_error, E_2_heating_error)

# Ochładzanie
E_1_cooling_error = round_sig(
    2 * boltzmann_constant * a_1_cooling_error * joule_to_electronvolt, 2
)
E_2_cooling_error = round_sig(
    2 * boltzmann_constant * a_2_cooling_error * joule_to_electronvolt, 2
)

print("Ochladzanie")
print(E_1_cooling_error, E_2_cooling_error)


### Obliczenie niepewności rozszerzonej

In [ ]:
# wzor = U = 2sqrt(u^2 + u^2)
# 2 niepewnosci dla ochladzania i dla ocieplania

extended_uncertainty_heating = 2 * sqrt(E_1_heating_error**2 + E_2_heating_error**2)
extended_uncertainty_cooling = 2 * sqrt(E_1_cooling_error**2 + E_2_cooling_error**2)

print(round_sig(extended_uncertainty_heating, 2))
print(round_sig(extended_uncertainty_cooling, 2))

print("Sprawdzenie czy termistory sa jednakowe")
print(
    round_sig(abs(E_1_heating - E_2_heating), 3)
    < round_sig(extended_uncertainty_heating, 3)
)
print(
    round_sig(abs(E_1_cooling - E_2_cooling), 3)
    < round_sig(extended_uncertainty_cooling, 3)
)
